In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
import optuna
from sklearn.metrics import mean_absolute_percentage_error as MAPE
from sklearn.metrics import mean_squared_error 

import lightgbm as lgbm
import joblib
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import optuna 
from optuna.visualization import plot_optimization_history, plot_param_importances
from optuna import Trial, visualization
from optuna.samplers import TPESampler

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from statistics import mean
from s3_utils.s3_utils import write_parquet_from_pd, read_pd_from_parquet, start_logger,write_pickle,read_pickle

from s3_utils import read_pd_from_csv

import gc
pd.set_option('display.max_rows', 500)

ModuleNotFoundError: No module named 'optuna'

In [3]:
from data_validation import quality_check,missing_threshold,df_optimized

In [4]:
prospectos = pd.read_csv("s3://data-science-kavak-dev/projects/cerberus/v2/dev/income/data/prospectos.csv", index_col=0)
prospectos.shape

ImportError: Install s3fs to access S3

In [4]:
aprobados = pd.read_csv("s3://data-science-kavak-dev/projects/cerberus/v2/dev/income/data/aprobados.csv", usecols=prospectos.columns)
aprobados.shape

(13943, 50)

In [37]:
df = prospectos.append(aprobados, ignore_index=True)
del prospectos
del aprobados
gc.collect()

51

In [39]:
df = df_optimized(df)

optimized size by 52.0 % | 0.005228498 GB


In [40]:
quality_check(df)

Evaluation uniques an Nan by Column


,column_name,percent_missing,percent_unique,sample_values
45,nombre del score,100.000000,0.000000,[nan]
7,nueva direccion en los últimos 60 días,100.000000,0.000000,[nan]
1,declarativa,100.000000,0.000000,[nan]
5,mensaje de alerta,100.000000,0.000000,[nan]
6,moneda del credito,100.000000,0.000000,[nan]
43,código de razón 3,69.771133,0.066445,"[nan, 21.0, 5.0, 31.0, 9.0, 32.0, 4.0, 28.0, 2..."
42,código de razón 2,35.282392,0.066445,"[nan, 9.0, 16.0, 28.0, 32.0, 4.0, 5.0, 15.0, 1..."
41,código de razón 1,6.530085,0.095976,"[4.0, 17.0, 28.0, 59.0, 15.0, 58.0, 5.0, 21.0,..."
3,fecha de apertura de la cuenta más reciente,3.606497,8.353636,"[-1630.0, -49.0, -352.0, -321.0, -71.0, -412.0..."
2,fecha de apertura de la cuenta más antigua,3.606497,29.198966,"[-2977.0, -2353.0, -997.0, -4237.0, -2041.0, -..."


In [15]:
missing_threshold(df, 0.69)

Original shape (27090, 50)
Final shape (27090, 43)


In [19]:
df.columns

Index(['researchable_id', 'fecha de apertura de la cuenta más antigua',
       'fecha de apertura de la cuenta más reciente',
       'fecha de la consulta mas reciente', 'número de cuentas',
       'número de cuentas cerradas',
       'número de cuentas con historial de morosidad',
       'número de cuentas con mop = 00', 'número de cuentas con mop = 01',
       'número de cuentas con mop = 02', 'número de cuentas con mop = 03',
       'número de cuentas con mop = 04', 'número de cuentas con mop = 05',
       'número de cuentas con mop = 06', 'número de cuentas con mop = 07',
       'número de cuentas con mop = 96', 'número de cuentas con mop = 97',
       'número de cuentas con mop = 99', 'número de cuentas con mop = UR',
       'número de cuentas con morosidad actual',
       'número de cuentas de pagos fijos e hipotecarios',
       'número de cuentas en aclaración',
       'número de cuentas en despacho de cobranza o administadora de cartera',
       'número de cuentas revolventes y

In [22]:
to_drop = ["estimate",'researchable_id']
model_cols = [col for col in df.columns if col not in to_drop]
model_df = df[model_cols]

In [23]:
quality_check(model_df)

Evaluation uniques an Nan by Column


,column_name,percent_missing,percent_unique,sample_values
36,código de razón 1,6.530085,0.095976,"[4.0, 17.0, 28.0, 59.0, 15.0, 58.0, 5.0, 21.0,..."
0,fecha de apertura de la cuenta más antigua,3.606497,29.198966,"[-2977.0, -2353.0, -997.0, -4237.0, -2041.0, -..."
1,fecha de apertura de la cuenta más reciente,3.606497,8.353636,"[-1630.0, -49.0, -352.0, -321.0, -71.0, -412.0..."
2,fecha de la consulta mas reciente,1.343669,0.003691,"[0.0, nan]"
15,número de cuentas con mop = 97,0.579550,0.036914,"[0.0, 1.0, 2.0, nan, 3.0, 4.0, 6.0, 8.0, 5.0, ..."
25,número de solicitudes del informe de buró real...,0.579550,0.003691,"[0.0, nan]"
24,número de solicitudes del informe de buró,0.579550,0.365449,"[18.0, 28.0, 14.0, 17.0, 25.0, 39.0, 35.0, 22...."
23,número de solicitudes de consulta,0.579550,0.262089,"[9.0, 12.0, 4.0, 8.0, 2.0, 6.0, 7.0, 16.0, 10...."
22,número de cuentas revolventes y sin límite pre...,0.579550,0.166113,"[4.0, 16.0, 2.0, 25.0, 6.0, 14.0, 3.0, 12.0, 1..."
21,número de cuentas en despacho de cobranza o ad...,0.579550,0.018457,"[0.0, nan, 1.0, 3.0, 2.0, 4.0]"


In [25]:
model_df = model_df.query("net_income_verified > 0 & net_income_verified < 500000")

mop > 4 o MP > 96 o MOP>04 o MO = UR

In [32]:
mops = [col for col in model_df if "mop" in col]
mops

['número de cuentas con mop = 00',
 'número de cuentas con mop = 01',
 'número de cuentas con mop = 02',
 'número de cuentas con mop = 03',
 'número de cuentas con mop = 04',
 'número de cuentas con mop = 05',
 'número de cuentas con mop = 06',
 'número de cuentas con mop = 07',
 'número de cuentas con mop = 96',
 'número de cuentas con mop = 97',
 'número de cuentas con mop = 99',
 'número de cuentas con mop = UR']

In [31]:
mayor_a_4 = ['número de cuentas con mop = 04',
 'número de cuentas con mop = 05',
 'número de cuentas con mop = 06',
 'número de cuentas con mop = 07']

incobrable = ['número de cuentas con mop = 96','número de cuentas con mop = 97','número de cuentas con mop = 99']

model_df["mayor_a_4"] = model_df.apply(lambda x: 1 if x[mayor_a_4].any()>0 else 0, axis=1)
model_df["incobrable"] = model_df.apply(lambda x: 1 if x[incobrable].any()>0 else 0, axis=1)
model_df["sin_datos_de_payments"] = model_df['número de cuentas con mop = UR']

In [34]:
select_cols = [col for col in model_df.columns if col not in mops ]
model_df = model_df[select_cols]

In [35]:
model_df

,fecha de apertura de la cuenta más antigua,fecha de apertura de la cuenta más reciente,fecha de la consulta mas reciente,número de cuentas,número de cuentas cerradas,número de cuentas con historial de morosidad,número de cuentas con morosidad actual,número de cuentas de pagos fijos e hipotecarios,número de cuentas en aclaración,número de cuentas en despacho de cobranza o administadora de cartera,...,total de saldos vencidos para cuentas de pagos fijos e hipotecarios,total de saldos vencidos para cuentas revolventes y sin límite pre-establecido,código de razón 1,código del score,valor del score,net_income_verified,ingreso_neto_comprobado,mayor_a_4,incobrable,sin_datos_de_payments
0,-2977.0,-1630.0,0.0,4.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,4.0,7.0,724.0,22000.0,22000.0,0,0,0.0
1,-2353.0,-49.0,0.0,18.0,9.0,0.0,0.0,2.0,0.0,0.0,...,0.0,0.0,4.0,7.0,713.0,28000.0,28000.0,0,0,1.0
2,-997.0,-352.0,0.0,3.0,1.0,1.0,1.0,1.0,0.0,0.0,...,0.0,264.0,17.0,7.0,627.0,20000.0,18455.0,0,0,0.0
3,-997.0,-352.0,0.0,3.0,1.0,1.0,1.0,1.0,0.0,0.0,...,0.0,264.0,17.0,7.0,627.0,20000.0,18455.0,0,0,0.0
4,-4237.0,-321.0,0.0,29.0,13.0,3.0,1.0,4.0,0.0,0.0,...,0.0,4220.0,28.0,7.0,682.0,38000.0,38000.0,0,1,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27085,-2287.0,-167.0,0.0,10.0,3.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,21.0,7.0,658.0,30000.0,30000.0,0,0,1.0
27086,-6251.0,-59.0,0.0,5.0,1.0,1.0,1.0,1.0,0.0,0.0,...,0.0,40086.0,28.0,7.0,602.0,26000.0,26000.0,0,1,0.0
27087,-2940.0,-70.0,0.0,18.0,12.0,4.0,1.0,7.0,0.0,0.0,...,152784.0,0.0,28.0,7.0,601.0,300000.0,300000.0,0,1,4.0
27088,-2375.0,-2224.0,0.0,2.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,3852.0,21.0,7.0,692.0,15000.0,15000.0,0,1,0.0


In [26]:
list(model_df)

['fecha de apertura de la cuenta más antigua',
 'fecha de apertura de la cuenta más reciente',
 'fecha de la consulta mas reciente',
 'número de cuentas',
 'número de cuentas cerradas',
 'número de cuentas con historial de morosidad',
 'número de cuentas con mop = 00',
 'número de cuentas con mop = 01',
 'número de cuentas con mop = 02',
 'número de cuentas con mop = 03',
 'número de cuentas con mop = 04',
 'número de cuentas con mop = 05',
 'número de cuentas con mop = 06',
 'número de cuentas con mop = 07',
 'número de cuentas con mop = 96',
 'número de cuentas con mop = 97',
 'número de cuentas con mop = 99',
 'número de cuentas con mop = UR',
 'número de cuentas con morosidad actual',
 'número de cuentas de pagos fijos e hipotecarios',
 'número de cuentas en aclaración',
 'número de cuentas en despacho de cobranza o administadora de cartera',
 'número de cuentas revolventes y sin límite pre-establecido',
 'número de solicitudes de consulta',
 'número de solicitudes del informe de b